In [1]:
import gym
from tqdm import tqdm
#from .autonotebook import tqdm as notebook_tqdm
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy
from statistics import mean

/home/elena/PycharmProjects/esercitazioni_RL/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#define env
env_id = "CartPole-v1"
#env_id = "Acrobot-v1"
env = gym.make(env_id)

In [3]:
#define expert agent
ppo_expert = PPO('MlpPolicy', env_id, verbose=1, create_eval_env=True)

#train expert
ppo_expert.learn(total_timesteps=3e4, eval_freq=10000)

#save expert
ppo_expert.save("ppo_expert")

#evaluate expert
mean_reward, std_reward = evaluate_policy(ppo_expert, env, n_eval_episodes=10)
print(f"Mean reward expert agent= {mean_reward} +/- {std_reward}")

Using cpu device
Creating environment from the given name 'Acrobot-v1'
Creating environment from the given name 'Acrobot-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/run/user/1000/app/com.jetbrains.PyCharm-Community/ipykernel_1479/3493697535.py:2: DeprecationWarning: The parameter `create_eval_env` is deprecated and will be removed in the future. Please use `EvalCallback` or a custom Callback instead.
  ppo_expert = PPO('MlpPolicy', env_id, verbose=1, create_eval_env=True)
/run/user/1000/app/com.jetbrains.PyCharm-Community/ipykernel_1479/3493697535.py:5: DeprecationWarning: Parameters `eval_env` and `eval_freq` are deprecated and will be removed in the future. Please use `EvalCallback` or a custom Callback instead.
  ppo_expert.learn(total_timesteps=3e4, eval_freq=10000)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 500      |
|    ep_rew_mean     | -500     |
| time/              |          |
|    fps             | 1697     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 500         |
|    ep_rew_mean          | -500        |
| time/                   |             |
|    fps                  | 1261        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009083457 |
|    clip_fraction        | 0.0435      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.0217     |
|    learning_rate        | 0.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 249      |
|    ep_rew_mean     | -248     |
| time/              |          |
|    fps             | 973      |
|    iterations      | 10       |
|    time_elapsed    | 21       |
|    total_timesteps | 20480    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 230          |
|    ep_rew_mean          | -229         |
| time/                   |              |
|    fps                  | 974          |
|    iterations           | 11           |
|    time_elapsed         | 23           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0085445065 |
|    clip_fraction        | 0.0771       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.849       |
|    explained_variance   | 0.75         |
|    learning_r

/home/elena/PycharmProjects/esercitazioni_RL/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward expert agent= -121.4 +/- 85.71254283942345


In [4]:
##create expert dataset

num_interactions = int(4e4)

#define empty dataset
expert_observations = ...np...
expert_actions = ...np...


#collect experience usign expert policy
obs = env.reset()
for i in tqdm(range(num_interactions)):
    action, _ = ppo_expert.predict(obs, deterministic=True)
    
    #TODO: do a step
    
    expert_observations[i] = ...
    expert_actions[i] = ...
    
    if done:
        obs = env.reset()
        


(40000, 6)
(40000,)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:17<00:00, 2227.12it/s]


In [5]:
#save dataset
np.savez_compressed(
   "expert_data",
   expert_actions=expert_actions,
   expert_observations=expert_observations,
   )

In [6]:
##dataset class
from torch.utils.data.dataset import Dataset, random_split

class ExpertDataSet(Dataset):

    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions

    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

In [7]:
expert_dataset = ExpertDataSet(expert_observations, expert_actions)

#TODO: split in 80% training and 20%test
batch_size = 64
train_prop = 0.8

train_expert_dataset = ...
test_expert_dataset = ...

train_loader = th.utils.data.DataLoader(  dataset=train_expert_dataset, batch_size=batch_size, shuffle=True)
test_loader = th.utils.data.DataLoader(  dataset=test_expert_dataset, batch_size=batch_size, shuffle=True)

In [11]:
###### Define student agent
no_cuda = False
use_cuda = not no_cuda and th.cuda.is_available()
   
device = th.device("cuda" if use_cuda else "cpu")

class StudentAgent:
    def __init__(self, env, train_loader, test_loader, learning_rate):
        self.env = env
        self.train_loader = train_loader
        self.test_loader = test_loader

        #TODO: define a policy network
        self.policy = ...th...
        
        print("policy net: ", self.policy)
        
        #TODO: define loss criterion
        self.loss_criterion = ...
        
        #TODO: define optimizer
        self.optimizer =  ...
        
        self.num_eval_episodes = 10
        
    def train(self, num_epochs):
        self.policy.train()
        self.policy.to(device)
        for epoch in range(num_epochs):
            for batch_idx, (data, target) in enumerate(train_loader):
                obs, expert_action = data.to(device), target.to(device)
                self.optimizer.zero_grad()
                obs = obs.float()
                
                #TODO
                student_action = ...
                loss = ...
                
                loss.backward()
                self.optimizer.step()
            #compute accuracy
            train_acc = self.compute_accuracy(self.train_loader)
            test_acc = self.compute_accuracy(self.test_loader)
            policy_return = self.evaluate_policy(self.num_eval_episodes)
            print("Epoch {}:\ttrain accuracy: {}\ttest accuracy: {}\tpolicy return:{}".format(epoch, train_acc, test_acc, policy_return))

    def compute_accuracy(self, loader):
        total = 0
        correct = 0
        
        self.policy.eval()
        test_loss = 0
        with th.no_grad():
            for data, target in test_loader:
                obs, expert_action = data.to(device), target.to(device)
                obs = obs.float()
                
                #TODO
                student_action = ...
            
                total += ...
                correct += ...
            
        accuracy = 100. * correct/(float)(total)
            
        return accuracy
            
    def policy_action(self, obs):
        #TODO
        policy_act = ...
        
        return policy_act
        
    def evaluate_policy(self, num_episodes, render=False):
        rewards = []
        for ep in range(num_episodes):
            done = False
            tot_rew = 0
            obs = self.env.reset()

            while not done:
                #TODO
                action = ...
                
                obs, reward, done, info = env.step(action)
                if render:
                    env.render()
                tot_rew += reward
            rewards.append(tot_rew)
        return mean(rewards)
    

            

    

In [12]:
student = StudentAgent(env, train_loader, test_loader, 0.01)
student.train(50)

policy net:  Sequential(
  (0): Linear(in_features=6, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=3, bias=True)
  (3): Softmax(dim=-1)
)
Epoch 0:	train accuracy: 99.675	test accuracy: 99.675	policy return:-86.5
Epoch 1:	train accuracy: 99.6125	test accuracy: 99.6125	policy return:-79.3
Epoch 2:	train accuracy: 99.675	test accuracy: 99.675	policy return:-84.5
Epoch 3:	train accuracy: 99.5375	test accuracy: 99.5375	policy return:-76.7
Epoch 4:	train accuracy: 99.675	test accuracy: 99.675	policy return:-78.8
Epoch 5:	train accuracy: 99.7625	test accuracy: 99.7625	policy return:-79.3
Epoch 6:	train accuracy: 99.775	test accuracy: 99.775	policy return:-81.8
Epoch 7:	train accuracy: 99.4	test accuracy: 99.4	policy return:-88.9
Epoch 8:	train accuracy: 99.6875	test accuracy: 99.6875	policy return:-86.6
Epoch 9:	train accuracy: 99.6875	test accuracy: 99.6875	policy return:-82.3
Epoch 10:	train accuracy: 99.7625	test accuracy: 99.7625	policy return:-80.9